# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.vfm.connection import Connection
from src.vfm.preprocessor import Preprocessor
import os
import sys

connection = Connection()

# Add the parent directory to the sys.path list
sys.path.insert(0, os.path.abspath('..'))

# Get client
client = connection.get_client()


In [3]:
well = 'W06'
df_raw = connection.get_data(client, well = well, start='2019-01-01', end='2022-04-18')

In [6]:
preprocessor = Preprocessor(df_raw=df_raw)
df = preprocessor.preprocess()

In [7]:
from sklearn.model_selection import train_test_split

# Select all independent variables.
X = df[['dhp', 'dht', 'whp', 'wht', 'choke', 'dcp']]
# Select only the target variable.
Y = df[['qo_mpfm', 'qg_mpfm', 'qw_mpfm']]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=65)

In [8]:
# Training data
print(f'Train data records  {X_train.shape}')

# Testing data
print(f'Test data records {X_test.shape}')

Train data records  (511340, 7)
Test data records (127835, 7)


# Feed Forward Neural Network Model - Well 06

In [9]:
from src.vfm.model.nn.feed_forward import FeedForwardNeuralNetworkEvaluator
ff_nn = FeedForwardNeuralNetworkEvaluator(X_train, Y_train, X_test, Y_test)
ff_nn.train()

/home/priyanka/.cache/pypoetry/virtualenvs/vfm-nn-DOmiObSy-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2023-10-30 16:00:30,402] A new study created in memory with name: no-name-d5d8ec11-4840-43fc-9e80-fb5f71fcc72f
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                                          | Type       | Params
-----------------------------------------------------------------------------
0 | _FeedForwardNeuralNetworkModel__criterion     | MSELoss    | 0     
1 | _FeedForwardNeuralNetworkModel__fc1           | Linear     | 416   
2 | _FeedForwardNeuralNetworkModel__hidden_layers | ModuleList | 44.1 K
3 | _FeedForwardNeuralNet

num_hidden_layers = 16, num_hidden_features = 52, learning_rate = 1.385169292512366e-05, num_epochs = 128
Epoch 1:  74%|███████▍  | 5925/7990 [00:25<00:08, 236.78it/s, v_num=26]

/home/priyanka/.cache/pypoetry/virtualenvs/vfm-nn-DOmiObSy-py3.10/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
[I 2023-10-30 16:01:30,223] Trial 0 finished with value: 1930668.375 and parameters: {'num_hidden_layers': 16, 'num_hidden_features': 52, 'learning_rate': 1.385169292512366e-05, 'num_epochs': 128}. Best is trial 0 with value: 1930668.375.


num_hidden_layers = 19, num_hidden_features = 58, learning_rate = 1.4743924513056339e-05, num_epochs = 112


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/priyanka/.cache/pypoetry/virtualenvs/vfm-nn-DOmiObSy-py3.10/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:615: UserWarning: Checkpoint directory /home/priyanka/Documents/GitHub/vfm-nn/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                                          | Type       | Params
-----------------------------------------------------------------------------
0 | _FeedForwardNeuralNetworkModel__criterion     | MSELoss    | 0     
1 | _FeedForwardNeuralNetworkModel__fc1           | Linear     | 464   
2 | _FeedForwardNeuralNetworkModel__hidden_layers | ModuleList | 65.0 K
3 | _FeedForwardNeuralNetworkModel__fc3           | Linear     | 177   
--------------------------------

...

[I 2023-10-30 16:15:02,251] Trial 3 finished with value: 1555950.875 and parameters: {'num_hidden_layers': 11, 'num_hidden_features': 48, 'learning_rate': 0.0002466035503728178, 'num_epochs': 105}. Best is trial 1 with value: 1357786.75.


num_hidden_layers = 15, num_hidden_features = 43, learning_rate = 1.7018584141620544e-05, num_epochs = 105


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                                          | Type       | Params
-----------------------------------------------------------------------------
0 | _FeedForwardNeuralNetworkModel__criterion     | MSELoss    | 0     
1 | _FeedForwardNeuralNetworkModel__fc1           | Linear     | 344   
2 | _FeedForwardNeuralNetworkModel__hidden_layers | ModuleList | 28.4 K
3 | _FeedForwardNeuralNetworkModel__fc3           | Linear     | 132   
-----------------------------------------------------------------------------
28.9 K    Trainable params
0         Non-trainable params
28.9 K    Total params
0.115     Total estimated model params size (MB)


Epoch 2:  87%|████████▋ | 6990/7990 [00:28<00:04, 245.85it/s, v_num=30]

[I 2023-10-30 16:16:32,367] Trial 4 finished with value: 2355182.0 and parameters: {'num_hidden_layers': 15, 'num_hidden_features': 43, 'learning_rate': 1.7018584141620544e-05, 'num_epochs': 105}. Best is trial 1 with value: 1357786.75.


In [10]:
ff_nn.test()

Best model :
num_hidden_layers = 19, num_hidden_features = 58, learning_rate = 1.4743924513056339e-05, num_epochs = 112


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0:  22%|██▏       | 431/1998 [00:00<00:02, 559.04it/s]

TypeError: 'NoneType' object is not subscriptable

In [ ]:
ff_nn.results()